<a href="https://colab.research.google.com/github/PradyumnaKrishna/Colab-Hacks/blob/RDP-v2/Colab%20RDP/Colab%20RDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab RDP** : Remote Desktop to Colab Instance

Used Google Remote Desktop & Ngrok Tunnel

> **Warning :** This notebook is against the Policy of Colab. Use it on your own risk

Google Colab can give you Instance with 12GB of RAM and GPU for 12 hours (Max.). Anyone can use it to perform Heavy Tasks

To use other similiar Notebooks use my Repository **[Colab Hacks](https://github.com/PradyumnaKrishna/Colab-Hacks)**

In [ ]:
#@title **One Click RDP**
#@markdown  It takes 4-5 minutes for installation

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

#@markdown  Visit http://remotedesktop.google.com/headless and Copy the command after authentication

CRP = "" #@param {type:"string"}

#@markdown  To Sleep the Colab Notebook

Sleep = True #@param {type:"boolean"}


print("Creating User and Setting it up")

# Creation of user
! sudo useradd -m $username &> /dev/null

# Add user to sudo group
! sudo adduser $username sudo &> /dev/null
    
# Set password of user to 'root'
! echo '$username:$password' | sudo chpasswd

# Change default shell from sh to bash
! sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd

print("User Created and Configured")

with open('install.sh', 'w') as script:
    script.write("""#! /bin/bash

b='\033[1m'
r='\E[31m'
g='\E[32m'
c='\E[36m'
endc='\E[0m'
enda='\033[0m'

printf "\n\n$c$b    Loading Installer $endc$enda" >&2
if sudo apt-get update &> /dev/null
then
    printf "\r$g$b    Installer Loaded $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
    exit
fi

printf "\n$g$b    Installing Chrome Remote Desktop $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb
    sudo dpkg --install chrome-remote-desktop_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Chrome Remote Desktop Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Desktop Environment $endc$enda" >&2
{
    sudo DEBIAN_FRONTEND=noninteractive \
        apt install --assume-yes xfce4 desktop-base
    sudo bash -c 'echo "exec /etc/X11/Xsession /usr/bin/xfce4-session" > /etc/chrome-remote-desktop-session'  
    sudo apt install --assume-yes xscreensaver
    sudo systemctl disable lightdm.service
} &> /dev/null &&
printf "\r$c$b    Desktop Environment Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Google Chrome $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    sudo dpkg --install google-chrome-stable_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Google Chrome Installed $endc$enda\n" >&2 ||
printf "\r$r$b    Error Occured $endc$enda\n" >&2
sleep 3

printf "$g$b    Installing other Tools $endc$enda" >&2
if sudo apt install nautilus nano -y &> /dev/null
then
    printf "\r$c$b    Other Tools Installed $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
fi
sleep 3

printf "\n$g$b    Installation Completed $endc$enda\n\n" >&2""")
    
! chmod +x install.sh
! ./install.sh

# Adding user to CRP group
! sudo adduser $username chrome-remote-desktop &> /dev/null

# Finishing Work
! su - $username -c """$CRP"""

print("Finished Succesfully")

if Sleep:
    ! sleep 43200

In [ ]:
#@title Google Drive Mount
#@markdown Google Drive used as Persistance HDD for files.<br>
#@markdown Mounted at `user` Home directory inside drive folder.

#@markdown Type username or leave default

username = "user" #@param {type:"string"}
mountpoint = "/home/"+username+"/drive"

from os import environ as env
from os.path import join as path_join
from google.colab import drive

env['user'] = username
env['user_home'] = "/home/{}".format(username)
env['mount_script'] = path_join(env['user_home'], 'drive_mount.sh')

! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

! cp -r /root/.config/Google /home/$user/.config/Google > /dev/null 2>&1
! mkdir -p /home/$user/.config/Google/DriveFS/Logs
! chown -R $user:$user /home/user/.config/Google/DriveFS/Logs
! mkdir -p $mountpoint > /dev/null 2>&1
! chown -R $user:$user /content/.config
! chown -R $user:$user /content > /dev/null 2>&1

mount = """
#!/bin/sh

from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'
drive.mount('{}')
""".format(mountpoint)

with open(path_join(env['user_home'], "drive_mount.sh"), "w") as script:
    script.write(mount)

! chown -R  $user $user_home
! chmod u+x $mount_script

! runuser -l $user -c "python3 /home/$user/drive_mount.sh"

! chown $user:$user $mountpoint > /dev/null 2>&1

## **Standard Method**

### Before We Started
- Create a Ngrok Account to configure tunnel
- Don't use username : `root`

### Installing Colab SSH Library

In [ ]:
! pip install colab_ssh --upgrade
from colab_ssh import launch_ssh, init_git

### Configure User

Setting Username & Password

In [ ]:
username = input("Enter username : ")
password = input("Enter password : ")

Create User & Set Password

In [ ]:
! sudo useradd -m $username
! sudo adduser $username sudo
! echo '$username:$password' | sudo chpasswd
! sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd

### Mount Google Drive

In [ ]:
from google.colab import drive
home='/home/'+username+'/drive'
drive.mount(home)

### Setting Chrome Remote Desktop

Download Remote Desktop Installation Script

In [ ]:
! wget https://github.com/PradyumnaKrishna/Colab-Hacks/raw/master/Colab%20RDP/install.sh -P /home/$username

Use the script 

- It install the required files and configure your instance
- It takes 4-5 mintues to complete the process

In [ ]:
! sudo chmod +x /home/$username/install.sh
! sudo /home/$username/install.sh

### Run Ngrok Tunnel

In [ ]:
print("Copy authtoken from https://dashboard.ngrok.com/auth")
ngrokToken = input("Enter Token : ")

launch_ssh(ngrokToken, password)

Connect to instance via SSH 
- Use the command below in your terminal
- When ask for password give your password which is set previously

In [ ]:
print("ssh", username, end='@')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"

If unable to Connect through SSH then Kill and Start NGROK Tunnel again

### Authenticate Through [Remote Desktop Headless](http://remotedesktop.google.com/headless)
- Open browser and go to http://remotedesktop.google.com/headless
- Dont Download any file simply proceed directly to authenticate button
- when you got a command copy and paste it to SSH terminal and complete the process by setting up a pin

Now your process completes 

Visit [Remote Desktop](http://remotedesktop.google.com/access) at http://remotedesktop.google.com/headless to access your instance

### Kill Ngrok Tunnel

In [ ]:
! killall ngrok

### Sleep Colab
Sleep Colab for 12 hours

In [ ]:
! sleep 43200